In [1]:
# %pip install --upgrade langsmith

import os

print("Langchain key to start tracing")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-aching-poisoning-52"
os.environ["LANGCHAIN_API_KEY"] = "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"

Langchain key to start tracing


In [2]:
from getpass import getpass
import boto3

AWS_ACCESS_KEY = getpass("AWS Acces key: ")
AWS_SECRET_KEY = getpass("AWS Secret key: ")
AWS_REGION = "ap-south-1"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")


sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(client=bedrock_client,model_id="anthropic.claude-3-sonnet-20240229-v1:0")

c:\Users\361DT\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_arn" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [6]:
llm.invoke("Hi").content

Error raised by bedrock service
Traceback (most recent call last):
  File "c:\Users\361DT\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_aws\llms\bedrock.py", line 949, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\361DT\AppData\Local\Programs\Python\Python312\Lib\site-packages\botocore\client.py", line 570, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\361DT\AppData\Local\Programs\Python\Python312\Lib\site-packages\botocore\context.py", line 124, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\361DT\AppData\Local\Programs\Python\Python312\Lib\site-packages\botocore\client.py", line 1031, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ThrottlingException: An error 

ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.

In [7]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "For how many customers, company value is missing?"}).split('SQLQuery: ')[-1]
response

'\nSELECT COUNT(*) AS "Number of Customers with Missing Company"\nFROM "Customer"\nWHERE "Company" IS NULL;'

In [8]:
db.run(response)

'[(49,)]'

In [40]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result